In [9]:
#!git clone https://github.com/ArthurSSS9966/SoftHebb.git
#%cd SoftHebb

import os.path
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim

# Test to load the SoftHebb package
from dataset import make_data_loaders
from model import load_layers, HebbianOptimizer, AggregateOptim, save_layers
from engine import train_sup, evaluate_sup
from train import check_dimension, training_config
from utils import seed_init_fn, load_presets, load_config_dataset, CustomStepLR
from log import Log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

params = {"preset":"2SoftMlpMNIST", "dataset_sup":"MNIST", "dataset_unsup":"MNIST",
          "seed":52,"model-name":"2SoftMlpMNIST", "training_mode":"simultaneous", "training_blocks":None,
          "resume":False, "save":False}

name_model = params["preset"]
blocks = load_presets(params["preset"])
CNN_blocks = load_presets("2SoftHebbCnnCIFAR")

dataset_sup_config = load_config_dataset(params["dataset_sup"], 0.8)
dataset_unsup_config = load_config_dataset(params["dataset_unsup"], 0.8)
dataset_sup_config['validation'] = True

blocks = check_dimension(blocks, dataset_sup_config)
CNN_blocks = check_dimension(CNN_blocks, dataset_sup_config)

train_config = training_config(blocks, dataset_sup_config, dataset_unsup_config, params["training_mode"],
                               params["training_blocks"])

cnn_train_config = training_config(CNN_blocks, dataset_sup_config, dataset_unsup_config, params["training_mode"],
                                   params["training_blocks"])

config = train_config['t1']

train_loader, val_loader, test_loader = make_data_loaders(dataset_sup_config, config['batch_size'], device)


range = 2.165063509461097
range = 0.3872983346207417
block 0, size : 96 14 14
range = 5.0
range = 0.12626906806902635


D:\BlcRepo\OtherCode\NeuroAI\SoftHebb\dataset.py:694: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(self.data, dtype=torch.float, device=device).div_(255).unsqueeze(1)
D:\BlcRepo\OtherCode\NeuroAI\SoftHebb\dataset.py:696: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(self.targets, device=device)


In [3]:
SoftHebb_model = load_layers(blocks, name_model, False)
SoftHebb_CNN_model = load_layers(CNN_blocks, "2SoftHebbCnnCIFAR", False)


 Model 2SoftMlpMNIST not found



 ----- Architecture Block FlattenH78420002, number 0 -----
- Flatten(start_dim=1, end_dim=-1)
- HebbSoftLinear(in_features=784, out_features=2000, lebesgue_p=2,  bias=False, t_invert=12.0, bias=False, lr_bias=0.0833)
- SoftMax(t_invert=5.0, dim=1)

 ----- Architecture Block Linear(in_, number 1 -----
- Linear(in_features=2000, out_features=10, bias=True)

 Model 2SoftHebbCnnCIFAR not found



 ----- Architecture Block BatchNorm2dSK1962(5, 5)1.0reflect, number 0 -----
- BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
- HebbSoftKrotovConv2d(1, 96, lebesgue_p=2, pruning=0, kernel_size=(5, 5), bias=False, padding_mode=reflect, t_invert=1.0, bias=False, lr_bias=0.1, ranking_param=3, delta=2, activation=exp)
- Triangle(power=0.7)
- AvgPool2d(kernel_size=4, stride=2, padding=1)

 ----- Architecture Block FlattenDropout(p=0.5, inplace=False)Linear(in_, number 1 -----
- Flatten(start_dim=1, end_dim=-1)
- Dropout(p=0.5, inplace=F

In [4]:
def train_model(
        final_epoch: int,
        print_freq: int,
        lr: float,
        folder_name: str,
        model,
        device,
        log,
        blocks,
        learning_mode: str = 'BP'
):
    """
    Hybrid training of one model, happens during simultaneous training mode
    """
    print('\n', '********** Hybrid learning of blocks %s **********' % blocks)
    optimizer_sgd = optim.Adam(
        model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    hebbian_optimizer = HebbianOptimizer(model)
    scheduler = CustomStepLR(optimizer_sgd, final_epoch)
    optimizer = AggregateOptim((hebbian_optimizer, optimizer_sgd))
    log_batch = log.new_log_batch()
    
    val_accuracies = []
    
    for epoch in range(1, final_epoch + 1):
        measures, lr = train_sup(model, criterion, optimizer, train_loader, device, log_batch, learning_mode, blocks)

        if scheduler is not None:
            scheduler.step()

        if epoch % print_freq == 0 or epoch == final_epoch or epoch == 1:

            loss_test, acc_test = evaluate_sup(model, criterion, val_loader, device)
            
            log_batch = log.step(epoch, log_batch, loss_test, acc_test, lr, save=True)
            log.verbose()
            
            print(
                f'Loss test: {loss_test:.4f}, Accuracy test: {acc_test:.2f}%'
            )
            
            val_accuracies.append(acc_test)
        
        storing_path = os.path.join("D:\\BlcRepo\OtherCode\\NeuroAI\\SoftHebb\\Models", folder_name)
        
        if os.path.exists(storing_path):
            save_layers(model, folder_name, epoch, blocks, storing_path=storing_path)
        else:
            os.makedirs(storing_path)
            save_layers(model, folder_name, epoch, blocks, storing_path=storing_path)
            
    return log_batch, val_accuracies

In [5]:
log = Log(train_config)
config = train_config['t1']

SoftHebb_loss, SoftHebb_valloss = train_model(
                                                50,
                                                10,
                                                config['lr'],
                                                name_model,
                                                SoftHebb_model,
                                                device,
                                                log.sup['t1'],
                                                blocks=config['blocks']
                                            )


 ********** Hybrid learning of blocks [0, 1] **********
Epoch: [1/100]	lr: 1.00e-03	time: 00:00:12	Loss_train 0.03505	Acc_train 40.92	/	Loss_test 0.03424	Acc_test 52.28
Loss test: 0.0342, Accuracy test: 52.28%
Epoch: [10/100]	lr: 1.00e-03	time: 00:01:39	Loss_train 0.01912	Acc_train 90.24	/	Loss_test 0.00749	Acc_test 96.35
Loss test: 0.0075, Accuracy test: 96.35%
Epoch: [20/100]	lr: 2.50e-04	time: 00:03:27	Loss_train 0.00504	Acc_train 96.59	/	Loss_test 0.00377	Acc_test 96.53
Loss test: 0.0038, Accuracy test: 96.53%
Epoch: [30/100]	lr: 1.25e-04	time: 00:05:23	Loss_train 0.00319	Acc_train 96.69	/	Loss_test 0.00300	Acc_test 96.62
Loss test: 0.0030, Accuracy test: 96.62%
Epoch: [40/100]	lr: 3.13e-05	time: 00:07:02	Loss_train 0.00281	Acc_train 96.73	/	Loss_test 0.00287	Acc_test 96.48
Loss test: 0.0029, Accuracy test: 96.48%
Epoch: [50/100]	lr: 7.81e-06	time: 00:08:42	Loss_train 0.00274	Acc_train 96.69	/	Loss_test 0.00287	Acc_test 96.54
Loss test: 0.0029, Accuracy test: 96.54%


In [6]:
criterion = nn.CrossEntropyLoss()
loss_test, acc_test = evaluate_sup(SoftHebb_model, criterion, test_loader, device)

print(f'Accuracy of the model on the test images: {acc_test:.2f}%')

Accuracy of the model on the test images: 96.38%


In [7]:
log = Log(cnn_train_config)
config = cnn_train_config['t1']

CnnHebb_loss, CnnHebb_valloss = train_model(
                                            50,
                                            10,
                                            0.001,
                                            "2SoftHebbCnnMNIST",
                                            SoftHebb_CNN_model,
                                            device,
                                            log.sup['t1'],
                                            blocks=config['blocks']
                                        )


 ********** Hybrid learning of blocks [0, 1] **********
Epoch: [1/100]	lr: 1.00e-03	time: 00:00:45	Loss_train 0.03145	Acc_train 87.23	/	Loss_test 0.01277	Acc_test 91.09
Loss test: 0.0128, Accuracy test: 91.09%
Epoch: [10/100]	lr: 1.00e-03	time: 00:06:35	Loss_train 0.00380	Acc_train 94.14	/	Loss_test 0.00120	Acc_test 97.56
Loss test: 0.0012, Accuracy test: 97.56%
Epoch: [20/100]	lr: 2.50e-04	time: 00:14:03	Loss_train 0.00168	Acc_train 96.74	/	Loss_test 0.00088	Acc_test 98.26
Loss test: 0.0009, Accuracy test: 98.26%
Epoch: [30/100]	lr: 1.25e-04	time: 00:20:30	Loss_train 0.00134	Acc_train 97.38	/	Loss_test 0.00084	Acc_test 98.33
Loss test: 0.0008, Accuracy test: 98.33%
Epoch: [40/100]	lr: 3.13e-05	time: 00:27:14	Loss_train 0.00117	Acc_train 97.63	/	Loss_test 0.00079	Acc_test 98.48
Loss test: 0.0008, Accuracy test: 98.48%
Epoch: [50/100]	lr: 7.81e-06	time: 00:33:56	Loss_train 0.00112	Acc_train 97.76	/	Loss_test 0.00079	Acc_test 98.42
Loss test: 0.0008, Accuracy test: 98.42%


In [8]:
criterion = nn.CrossEntropyLoss()
loss_test, acc_test = evaluate_sup(SoftHebb_CNN_model, criterion, test_loader, device)

print(f'Accuracy of the model on the test images: {acc_test:.2f}%')

Accuracy of the model on the test images: 98.44%
